In [ ]:


import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, WeightedRandomSampler
from transformers import (
    AutoTokenizer, AutoModelForSequenceClassification,
    TrainingArguments, Trainer, EarlyStoppingCallback
)
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, 
    confusion_matrix, classification_report
)
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
import seaborn as sns
import json
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# Set random seeds
def set_seed(seed=42):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(42)

print("✓ All imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")


# ============================================================================
# SECTION 2: PREPROCESSING UTILITIES
# ============================================================================

import re
import emoji
import regex
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

NEPALI_STOPWORDS = set([
    "र", "मा", "कि", "भने", "त", "छ", "हो", "लाई", "ले",
    "गरेको", "गर्छ", "गर्छन्", "हुन्", "गरे", "न", "नभएको",
    "को", "का", "की", "ने", "पनि", "नै", "थियो", "थिए"
])

DIRGHIKARAN_MAP = {
    "उ": "ऊ", "इ": "ई", "ऋ": "रि", "ए": "ऐ", "अ": "आ",
    "\u200d": "", "\u200c": "", "।": ".", "॥": ".",
    "ि": "ी", "ु": "ू"
}

def is_devanagari(text: str) -> bool:
    if not isinstance(text, str) or not text.strip():
        return False
    return bool(regex.search(r'\p{Devanagari}', text))

def devanagari_to_roman(text: str) -> str:
    try:
        return transliterate(text, sanscript.DEVANAGARI, sanscript.ITRANS)
    except:
        return text

def roman_to_devanagari(text: str) -> str:
    try:
        return transliterate(text, sanscript.ITRANS, sanscript.DEVANAGARI)
    except:
        return text

def normalize_dirghikaran(text: str) -> str:
    for original, replacement in DIRGHIKARAN_MAP.items():
        text = text.replace(original, replacement)
    return text

def clean_text(text: str, aggressive: bool = False) -> str:
    if not isinstance(text, str):
        return ""
    
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", "", text, flags=re.MULTILINE)
    text = re.sub(r"@\w+|#\w+", "", text)
    text = emoji.replace_emoji(text, replace="")
    
    if aggressive:
        text = re.sub(r"\d+", "", text)
        text = re.sub(r"[^\w\s\u0900-\u097F]", "", text)
    
    text = re.sub(r"\s+", " ", text).strip()
    return text

def preprocess_for_transformer(text: str) -> str:
    if not isinstance(text, str):
        return ""
    
    if not is_devanagari(text):
        text = roman_to_devanagari(text)
    
    text = clean_text(text, aggressive=False)
    text = normalize_dirghikaran(text)
    
    return text

print("✓ Preprocessing utilities loaded")


# ============================================================================
# SECTION 3: FOCAL LOSS FOR HARD EXAMPLES
# ============================================================================

class FocalLoss(nn.Module):
    """
    Focal Loss: Focuses training on hard examples
    Paper: https://arxiv.org/abs/1708.02002
    
    Helps minority classes by down-weighting easy examples
    """
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha  # Class weights
        self.gamma = gamma  # Focusing parameter
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none', weight=self.alpha)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma) * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

print("✓ Focal Loss defined")


# ============================================================================
# SECTION 4: IMPROVED DATASET WITH STRATIFIED SAMPLING
# ============================================================================

class NepaliHateDataset(Dataset):
    """Dataset with better handling of class imbalance"""
    
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.texts)
    
    def __getitem__(self, idx):
        text = str(self.texts[idx]) if self.texts[idx] else ""
        label = self.labels[idx]
        
        encoding = self.tokenizer(
            text,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }


# ============================================================================
# SECTION 5: CUSTOM TRAINER WITH FOCAL LOSS & PER-CLASS MONITORING
# ============================================================================

class ImbalancedTrainer(Trainer):
    """
    Custom Trainer with:
    - Focal Loss
    - Per-class F1 monitoring
    - Better logging
    """
    
    def __init__(self, focal_loss_fn=None, label_names=None, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.focal_loss_fn = focal_loss_fn
        self.label_names = label_names or []
        self.training_history = {
            'train_loss': [],
            'eval_loss': [],
            'eval_f1_macro': [],
            'eval_f1_per_class': {name: [] for name in self.label_names}
        }
    
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        """
        Compute loss with support for both old and new transformers versions
        """
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        
        if self.focal_loss_fn is not None:
            loss = self.focal_loss_fn(logits, labels)
        else:
            loss = outputs.loss
        
        return (loss, outputs) if return_outputs else loss
    
    def prediction_step(self, model, inputs, prediction_loss_only, ignore_keys=None):
        """
        Override prediction_step to ensure we compute loss and get predictions
        """
        has_labels = "labels" in inputs
        inputs = self._prepare_inputs(inputs)
        
        with torch.no_grad():
            if has_labels:
                with self.compute_loss_context_manager():
                    loss, outputs = self.compute_loss(model, inputs, return_outputs=True)
                loss = loss.mean().detach()
            else:
                loss = None
                outputs = model(**inputs)
        
        if prediction_loss_only:
            return (loss, None, None)
        
        logits = outputs.logits
        labels = inputs.get("labels")
        
        return (loss, logits, labels)
    
    def evaluate(self, eval_dataset=None, ignore_keys=None, metric_key_prefix="eval"):
        """
        Override evaluate to ensure metrics are computed
        """
        eval_dataset = eval_dataset if eval_dataset is not None else self.eval_dataset
        eval_dataloader = self.get_eval_dataloader(eval_dataset)
        
        print(f"\n[DEBUG] Starting evaluation with {len(eval_dataset)} samples...")
        
        # Run predictions
        output = self.prediction_loop(
            eval_dataloader,
            description="Evaluation",
            prediction_loss_only=False,
            ignore_keys=ignore_keys,
            metric_key_prefix=metric_key_prefix,
        )
        
        print(f"[DEBUG] Prediction loop completed")
        print(f"[DEBUG] Output type: {type(output)}")
        print(f"[DEBUG] Has metrics attr: {hasattr(output, 'metrics')}")
        
        # Extract metrics
        if hasattr(output, 'metrics'):
            metrics = output.metrics
        else:
            metrics = output if isinstance(output, dict) else {}
        
        print(f"[DEBUG] Final metrics keys: {list(metrics.keys())}")
        
        # Ensure we have eval_loss
        if 'eval_loss' not in metrics and hasattr(output, 'loss'):
            metrics['eval_loss'] = output.loss
        
        self.log(metrics)
        self.control = self.callback_handler.on_evaluate(self.args, self.state, self.control, metrics)
        
        return metrics
    
    def log(self, logs, start_time=None):
        """
        Capture per-class metrics
        """
        # Debug: Print what we're logging
        if any('eval' in str(k) for k in logs.keys()):
            print(f"[DEBUG] Logging metrics: {list(logs.keys())}")
        
        super().log(logs)
        
        if 'loss' in logs:
            self.training_history['train_loss'].append(logs['loss'])
        if 'eval_loss' in logs:
            self.training_history['eval_loss'].append(logs['eval_loss'])
            print(f"[DEBUG] Captured eval_loss: {logs['eval_loss']:.4f}")
        if 'eval_f1' in logs:
            self.training_history['eval_f1_macro'].append(logs['eval_f1'])
            print(f"[DEBUG] Captured eval_f1: {logs['eval_f1']:.4f}")
        
        # Log per-class F1
        for i, name in enumerate(self.label_names):
            key = f'eval_f1_class_{i}'
            if key in logs:
                self.training_history['eval_f1_per_class'][name].append(logs[key])

print("✓ ImbalancedTrainer defined with enhanced evaluation")


def compute_metrics_detailed(eval_pred):
    """
    Compute metrics with per-class F1 scores
    """
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    
    # Debug: Check if function is being called
    print(f"\n[DEBUG] compute_metrics called with {len(labels)} samples")
    
    # Overall metrics
    accuracy = accuracy_score(labels, predictions)
    precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(
        labels, predictions, average='macro', zero_division=0
    )
    
    # Per-class F1
    precision_per_class, recall_per_class, f1_per_class, _ = precision_recall_fscore_support(
        labels, predictions, average=None, zero_division=0
    )
    
    metrics = {
        'accuracy': float(accuracy),
        'precision': float(precision_macro),
        'recall': float(recall_macro),
        'f1': float(f1_macro),
    }
    
    # Add per-class F1 to metrics
    for i, f1 in enumerate(f1_per_class):
        metrics[f'f1_class_{i}'] = float(f1)
    
    print(f"[DEBUG] Computed metrics: {list(metrics.keys())}")
    print(f"[DEBUG] F1 score: {f1_macro:.4f}")
    
    return metrics

print("✓ Custom Trainer and metrics defined")


# ============================================================================
# SECTION 6: DATA LOADING & AUGMENTATION
# ============================================================================

print("\n" + "="*70)
print(" LOADING DATA")
print("="*70)

# Load data
train_df = pd.read_json("/kaggle/input/nepalihate/train.json")
test_df = pd.read_json("/kaggle/input/nepalihate/test.json")

# Create validation split
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(
    train_df,
    test_size=0.15,
    stratify=train_df["Label_Multiclass"],
    random_state=42
)

print(f"✓ Data loaded:")
print(f"  Train: {len(train_df)}")
print(f"  Val:   {len(val_df)}")
print(f"  Test:  {len(test_df)}")

# Preprocess
print("\nPreprocessing texts...")
for df in [train_df, val_df, test_df]:
    df['transformer_input'] = df['Comment'].apply(preprocess_for_transformer)

print("✓ Preprocessing complete")

# Label encoding
le = LabelEncoder()
le.fit(train_df['Label_Multiclass'])

y_train = le.transform(train_df['Label_Multiclass'])
y_val = le.transform(val_df['Label_Multiclass'])
y_test = le.transform(test_df['Label_Multiclass'])

label_names = le.classes_.tolist()
num_classes = len(label_names)

print(f"\n✓ Labels encoded: {label_names}")
print(f"\nOriginal class distribution:")
for i, name in enumerate(label_names):
    count = (y_train == i).sum()
    print(f"  {name}: {count} ({count/len(y_train)*100:.2f}%)")

# CRITICAL: Apply oversampling for minority class
print("\n" + "="*70)
print(" APPLYING OVERSAMPLING FOR MINORITY CLASS")
print("="*70)

ros = RandomOverSampler(random_state=42, sampling_strategy='not majority')
X_train_texts = train_df['transformer_input'].values.reshape(-1, 1)
X_train_resampled, y_train_resampled = ros.fit_resample(X_train_texts, y_train)
X_train_resampled = X_train_resampled.flatten()

print(f"\nAfter oversampling:")
for i, name in enumerate(label_names):
    count = (y_train_resampled == i).sum()
    print(f"  {name}: {count} ({count/len(y_train_resampled)*100:.2f}%)")

print(f"\nTotal training samples: {len(y_train)} → {len(y_train_resampled)}")


# ============================================================================
# SECTION 7: COMPUTE ENHANCED CLASS WEIGHTS
# ============================================================================

print("\n" + "="*70)
print(" COMPUTING CLASS WEIGHTS")
print("="*70)

# Compute weights from original distribution (not resampled)
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(y_train),
    y=y_train
)

# BOOST minority class weight even more
minority_class_idx = np.argmin([np.sum(y_train == i) for i in range(num_classes)])
class_weights[minority_class_idx] *= 2.0  # Extra boost for OS class

# Cap extreme weights
class_weights = np.clip(class_weights, 0.5, 10.0)

print(f"\nClass weights:")
for i, name in enumerate(label_names):
    print(f"  {name}: {class_weights[i]:.2f}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float).to(device)

# Initialize Focal Loss
focal_loss = FocalLoss(alpha=class_weights_tensor, gamma=2.0)
print(f"✓ Focal Loss initialized (gamma=2.0)")


# ============================================================================
# SECTION 8: CREATE DATASETS & TOKENIZER
# ============================================================================

print("\n" + "="*70)
print(" INITIALIZING MODEL & TOKENIZER")
print("="*70)

model_name = 'xlm-roberta-base'

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    trust_remote_code=True,
    add_prefix_space=False
)
print(f"✓ Tokenizer loaded: {model_name}")

# Create datasets
train_dataset = NepaliHateDataset(
    X_train_resampled.tolist(),
    y_train_resampled,
    tokenizer,
    max_length=128
)

val_dataset = NepaliHateDataset(
    val_df['transformer_input'].tolist(),
    y_val,
    tokenizer,
    max_length=128
)

test_dataset = NepaliHateDataset(
    test_df['transformer_input'].tolist(),
    y_test,
    tokenizer,
    max_length=128
)

print(f"✓ Datasets created:")
print(f"  Train: {len(train_dataset)}")
print(f"  Val:   {len(val_dataset)}")
print(f"  Test:  {len(test_dataset)}")

# Load model
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_classes,
    problem_type="single_label_classification",
    ignore_mismatched_sizes=True
)

print(f"✓ Model loaded: {model_name}")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")


# ============================================================================
# SECTION 9: TRAINING CONFIGURATION
# ============================================================================

output_dir = '/kaggle/working/xlm_checkpoints'
os.makedirs(output_dir, exist_ok=True)

training_args = TrainingArguments(
    output_dir=output_dir,
    
    # Training
    num_train_epochs=4,  # More epochs for better learning
    per_device_train_batch_size=8,  # Smaller batch for better gradients
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=4,  # Effective batch size = 32
    
    # Optimization
    learning_rate=1e-5,  # Lower LR for stability
    warmup_ratio=0.15,
    weight_decay=0.01,
    max_grad_norm=1.0,
    lr_scheduler_type='cosine',
    
    # Regularization
    label_smoothing_factor=0.05,  # Lower smoothing
    
    # Evaluation - FIXED: Align eval and logging steps
    eval_strategy='steps',
    eval_steps=100,  # Evaluate every 100 steps
    save_strategy='steps',
    save_steps=100,
    save_total_limit=3,
    
    # Logging - CRITICAL: Must match eval_steps for validation loss to show
    logging_steps=100,  # Changed to match eval_steps
    logging_first_step=True,
    report_to='none',
    
    # Performance
    fp16=torch.cuda.is_available(),
    dataloader_num_workers=2,
    
    # Reproducibility
    seed=42,
    data_seed=42
)

print("✓ Training configuration ready (eval and logging aligned at 100 steps)")


# ============================================================================
# SECTION 10: INITIALIZE TRAINER & TRAIN
# ============================================================================

print("\n" + "="*70)
print(" STARTING TRAINING")
print("="*70)

trainer = ImbalancedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics_detailed,
    callbacks=[],  # Removed EarlyStoppingCallback to avoid metric issues
    focal_loss_fn=focal_loss,
    label_names=label_names
)

print(f"✓ Trainer initialized")
print(f"  Compute metrics function: {trainer.compute_metrics is not None}")
print(f"  Eval dataset size: {len(val_dataset)}")

# Train
train_result = trainer.train()

print("\n✓ Training complete!")
print(f"  Time: {train_result.metrics['train_runtime']:.2f}s")


# ============================================================================
# SECTION 11: SAVE MODEL
# ============================================================================

final_model_dir = '/kaggle/working/xlm_final'
os.makedirs(final_model_dir, exist_ok=True)

trainer.save_model(final_model_dir)
tokenizer.save_pretrained(final_model_dir)

import joblib
joblib.dump(le, os.path.join(final_model_dir, 'label_encoder.pkl'))

print(f"✓ Model saved to: {final_model_dir}")


# ============================================================================
# SECTION 12: EVALUATION & VISUALIZATION (CLEAN + FIXED)
# ============================================================================

print("\n" + "="*70)
print(" VALIDATION EVALUATION")
print("="*70)

# Run predictions
val_predictions = trainer.predict(val_dataset)
val_preds = np.argmax(val_predictions.predictions, axis=-1)
val_labels = np.array(val_predictions.label_ids)

# --- Compute per-class F1 once globally ---
precision_all, recall_all, f1_all, support_all = precision_recall_fscore_support(
    val_labels, val_preds, average=None, zero_division=0
)

print("\nPer-Class Validation Results:")
print("="*70)
for i, name in enumerate(label_names):
    mask = (val_labels == i)

    class_acc = (val_preds[mask] == i).sum() / mask.sum()
    class_f1 = f1_all[i]
    class_support = support_all[i]

    print(f"{name:20s}: "
          f"Accuracy={class_acc:.4f}, "
          f"F1={class_f1:.4f}, "
          f"Support={class_support}")

print("\n" + classification_report(
    val_labels, val_preds, target_names=label_names, digits=4
))

# --- Validation Confusion Matrix ---
plt.figure(figsize=(10, 8))
cm = confusion_matrix(val_labels, val_preds)
cm_normalized = cm.astype(float) / cm.sum(axis=1, keepdims=True)

sns.heatmap(cm_normalized, annot=True, fmt=".2%", cmap="Blues",
            xticklabels=label_names, yticklabels=label_names)
plt.title("Validation Confusion Matrix")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.savefig("/kaggle/working/val_confusion_matrix.png", dpi=300)
plt.show()



# ============================================================================
# TEST EVALUATION
# ============================================================================

print("\n" + "="*70)
print(" TEST EVALUATION")
print("="*70)

# Run predictions
test_predictions = trainer.predict(test_dataset)
test_preds = np.argmax(test_predictions.predictions, axis=-1)
test_labels = np.array(test_predictions.label_ids)

# Compute per-class once
precision_all_t, recall_all_t, f1_all_t, support_all_t = precision_recall_fscore_support(
    test_labels, test_preds, average=None, zero_division=0
)

print("\nPer-Class Test Results:")
print("="*70)
for i, name in enumerate(label_names):
    mask = (test_labels == i)

    class_acc = (test_preds[mask] == i).sum() / mask.sum()
    class_f1 = f1_all_t[i]
    class_support = support_all_t[i]

    print(f"{name:20s}: "
          f"Accuracy={class_acc:.4f}, "
          f"F1={class_f1:.4f}, "
          f"Support={class_support}")

print("\n" + classification_report(
    test_labels, test_preds, target_names=label_names, digits=4
))

# --- Test Confusion Matrix ---
plt.figure(figsize=(10, 8))
cm = confusion_matrix(test_labels, test_preds)
cm_normalized = cm.astype(float) / cm.sum(axis=1, keepdims=True)

sns.heatmap(cm_normalized, annot=True, fmt=".2%", cmap="Blues",
            xticklabels=label_names, yticklabels=label_names)
plt.title("Test Confusion Matrix")
plt.ylabel("True Label")
plt.xlabel("Predicted Label")
plt.tight_layout()
plt.savefig("/kaggle/working/test_confusion_matrix.png", dpi=300)
plt.show()



# ============================================================================
# TRAINING HISTORY VISUALIZATION
# ============================================================================

fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss plot
axes[0].plot(trainer.training_history['train_loss'], label='Train Loss', linewidth=2)

if trainer.training_history['eval_loss']:
    eval_steps = np.linspace(0, len(trainer.training_history['train_loss']) - 1,
                             len(trainer.training_history['eval_loss']))
    axes[0].plot(eval_steps, trainer.training_history['eval_loss'],
                 label='Val Loss', linewidth=2, marker='o')

axes[0].set_title("Training Loss")
axes[0].set_xlabel("Steps")
axes[0].set_ylabel("Loss")
axes[0].grid(True, alpha=0.3)
axes[0].legend()

# Macro F1 plot
if trainer.training_history['eval_f1_macro']:
    eval_steps = np.linspace(0, len(trainer.training_history['train_loss']) - 1,
                             len(trainer.training_history['eval_f1_macro']))
    axes[1].plot(eval_steps, trainer.training_history['eval_f1_macro'],
                 label='Macro F1', linewidth=2, marker='s')

axes[1].set_title("Validation F1 Score")
axes[1].set_xlabel("Steps")
axes[1].set_ylabel("F1 Score")
axes[1].grid(True, alpha=0.3)
axes[1].legend()

plt.tight_layout()
plt.savefig("/kaggle/working/training_history.png", dpi=300)
plt.show()


# ============================================================================
# SAVE FINAL TEST METRICS
# ============================================================================

test_accuracy = accuracy_score(test_labels, test_preds)
test_precision, test_recall, test_f1, _ = precision_recall_fscore_support(
    test_labels, test_preds, average="macro", zero_division=0
)

results = {
    "model": "XLM-RoBERTa-Fixed",
    "test_accuracy": float(test_accuracy),
    "test_precision": float(test_precision),
    "test_recall": float(test_recall),
    "test_f1": float(test_f1),
    "per_class_f1": {
        label_names[i]: float(f1_all_t[i]) for i in range(len(label_names))
    }
}

with open("/kaggle/working/results.json", "w") as f:
    json.dump(results, f, indent=2)

print("\n" + "="*70)
print(" FINAL RESULTS")
print("="*70)
print(f"Test Accuracy:  {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall:    {test_recall:.4f}")
print(f"Test Macro F1:  {test_f1:.4f}")
print("\nPer-Class F1:")
for name, f1 in results["per_class_f1"].items():
    print(f"  {name}: {f1:.4f}")
print("="*70)


print("\n🎉 Training complete! Check /kaggle/working/ for outputs")